# YNK forecast

## Import data

In [28]:
# Import data in csv from github

import pandas as pd

# Import data from github
url = 'https://raw.githubusercontent.com/sagebhardt/forecast-AI-ynk/main/Data_Cleaned.csv'
df = pd.read_csv(url)
df.head()

,Week,Store,Center,Type,Brand,SKU,SKU_Full,Description,Price,Denom.tipo mat.,...,Size,Season_Year,Season,PPL_Count,SUM_Q,Revenue,COGS,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,W09,01-Ahumada,1000,ZAPP,AD,ADHB0042,ADHB004200S,POLERON HODDIE DARK GREY T/ S,39990,Apparel,...,S,2022.0,Spring Summer,16519,1,24000,"20167,,,",NaN,NaN,NaN
1,W09,01-Ahumada,1000,ZAPP,AD,ADHE4926,ADHE492600L,POLERA WMS BASIC LOGO LIGH BLUE T / L,19990,Apparel,...,L,2022.0,Spring Summer,16519,1,19990,"10083,,,",NaN,NaN,NaN
2,W09,01-Ahumada,1000,ZAPP,AD,ADHK7092,ADHK709200M,SHORT BBALL T/M,24990,Apparel,...,M,2022.0,Fall,16519,1,24990,"12602,,,",NaN,NaN,NaN
3,W09,01-Ahumada,1000,ZAPP,AD,ADHL6934,ADHL693400L,POLERA M CAMO T/ L,22990,Apparel,...,L,2022.0,Fall,16519,1,22990,"11596,,,",NaN,NaN,NaN
4,W09,01-Ahumada,1000,ZAPP,AD,ADHR9501,ADHR950105T,POLERA LB DY SM T T/5T,24990,Apparel,...,5T,2023.0,Spring,16519,1,24990,"12602,,,",NaN,NaN,NaN


## Clean data  

In [29]:
#Delete last three columns
df = df.drop(df.columns[[-1, -2, -3]], axis=1)
df.head()

,Week,Store,Center,Type,Brand,SKU,SKU_Full,Description,Price,Denom.tipo mat.,...,Sub_Cat,Gender,Age_Group,Size,Season_Year,Season,PPL_Count,SUM_Q,Revenue,COGS
0,W09,01-Ahumada,1000,ZAPP,AD,ADHB0042,ADHB004200S,POLERON HODDIE DARK GREY T/ S,39990,Apparel,...,Performance,Male,Adult,S,2022.0,Spring Summer,16519,1,24000,"20167,,,"
1,W09,01-Ahumada,1000,ZAPP,AD,ADHE4926,ADHE492600L,POLERA WMS BASIC LOGO LIGH BLUE T / L,19990,Apparel,...,Performance,Female,Adult,L,2022.0,Spring Summer,16519,1,19990,"10083,,,"
2,W09,01-Ahumada,1000,ZAPP,AD,ADHK7092,ADHK709200M,SHORT BBALL T/M,24990,Apparel,...,Performance,Male,Adult,M,2022.0,Fall,16519,1,24990,"12602,,,"
3,W09,01-Ahumada,1000,ZAPP,AD,ADHL6934,ADHL693400L,POLERA M CAMO T/ L,22990,Apparel,...,Performance,Male,Adult,L,2022.0,Fall,16519,1,22990,"11596,,,"
4,W09,01-Ahumada,1000,ZAPP,AD,ADHR9501,ADHR950105T,POLERA LB DY SM T T/5T,24990,Apparel,...,Fashion,Male,Pre School,5T,2023.0,Spring,16519,1,24990,"12602,,,"


In [30]:
newdf = df['COGS'].map(lambda x: str(x)[:-3])
# Change the type of the column to float
df['COGS'] = newdf
df.head()


,Week,Store,Center,Type,Brand,SKU,SKU_Full,Description,Price,Denom.tipo mat.,...,Sub_Cat,Gender,Age_Group,Size,Season_Year,Season,PPL_Count,SUM_Q,Revenue,COGS
0,W09,01-Ahumada,1000,ZAPP,AD,ADHB0042,ADHB004200S,POLERON HODDIE DARK GREY T/ S,39990,Apparel,...,Performance,Male,Adult,S,2022.0,Spring Summer,16519,1,24000,20167
1,W09,01-Ahumada,1000,ZAPP,AD,ADHE4926,ADHE492600L,POLERA WMS BASIC LOGO LIGH BLUE T / L,19990,Apparel,...,Performance,Female,Adult,L,2022.0,Spring Summer,16519,1,19990,10083
2,W09,01-Ahumada,1000,ZAPP,AD,ADHK7092,ADHK709200M,SHORT BBALL T/M,24990,Apparel,...,Performance,Male,Adult,M,2022.0,Fall,16519,1,24990,12602
3,W09,01-Ahumada,1000,ZAPP,AD,ADHL6934,ADHL693400L,POLERA M CAMO T/ L,22990,Apparel,...,Performance,Male,Adult,L,2022.0,Fall,16519,1,22990,11596
4,W09,01-Ahumada,1000,ZAPP,AD,ADHR9501,ADHR950105T,POLERA LB DY SM T T/5T,24990,Apparel,...,Fashion,Male,Pre School,5T,2023.0,Spring,16519,1,24990,12602


In [26]:
# Get column types
df.dtypes

# Change the type of the column to int
newdf = df['COGS'].astype(int)

ValueError: invalid literal for int() with base 10: ''

# PCA Analysis